## Import

In [53]:
%load_ext autoreload
%autoreload 2

from initial_pop import *
from code_classes import *
from Operators.selection import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Population

In [7]:
class Player:
    def __init__(self, name, position, skill, cost):
        self.name = name
        self.position = position
        self.skill = skill
        self.cost = cost

    def __str__(self):
        return f"{self.name} ({self.position}) - Skill: {self.skill}, Cost: {self.cost}M"


In [15]:
class Team:
    def __init__(self, players):
        self.players = players  # players is a list of Player objects
        self.validate_team()

    def validate_team(self):
        positions = {"GK": 0, "DEF": 0, "MID": 0, "FWD": 0}
        for player in self.players:
            if player.position not in positions:
                raise ValueError(f"Invalid player position: {player.position}")
            positions[player.position] += 1

        # Check the required structure
        if positions["GK"] != 1 or positions["DEF"] != 2 or positions["MID"] != 2 or positions["FWD"] != 2:
            raise ValueError("Each team must have 1 GK, 2 DEF, 2 MID, and 2 FWD.")

        # Check if the team exceeds salary cap
        total_salary = sum(player.cost for player in self.players)
        if total_salary > 750:
            raise ValueError(f"Team salary exceeds the cap: {total_salary}M")

    def get_average_skill(self):
        return sum(player.skill for player in self.players) / len(self.players)

    def __str__(self):
        return "\n".join([str(player) for player in self.players])

In [8]:
class League:
    def __init__(self, teams):
        self.teams = teams
        self.validate_league()
                
    def validate_league(self):
        if len(self.teams) != 5:
            raise ValueError("The league must have exactly 5 teams.")

        player_names = set()
        for team in self.teams:
            # Explicitly call team.validate_team()
            team.validate_team()

            for player in team.players:
                if player.name in player_names:
                    raise ValueError(f"Player {player.name} is already in another team.")
                player_names.add(player.name)


    def is_valid(self):
        try:
            self.validate_league()
            return True
        except ValueError:
            return False

    def get_standard_deviation_of_average_skills(self):
        avg_skills = [team.get_average_skill() for team in self.teams]
        return np.std(avg_skills)

    def __str__(self):
        return "\n\n".join([str(team) for team in self.teams])

In [9]:
def calculate_fitness(league):
    if league is None or not league.is_valid():
        return 9999
    return league.get_standard_deviation_of_average_skills()

In [10]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

In [11]:
import random

def create_valid_team_from_pool(player_pool):
    max_attempts = 100
    for _ in range(max_attempts):
        gks = [p for p in player_pool if p.position == "GK"]
        defs = [p for p in player_pool if p.position == "DEF"]
        mids = [p for p in player_pool if p.position == "MID"]
        fwds = [p for p in player_pool if p.position == "FWD"]

        if len(gks) < 1 or len(defs) < 2 or len(mids) < 2 or len(fwds) < 2:
            raise ValueError("Not enough players in the pool to form a valid team.")

        selected_players = random.sample(gks, 1) + \
                           random.sample(defs, 2) + \
                           random.sample(mids, 2) + \
                           random.sample(fwds, 2)

        try:
            team = Team(selected_players)
            return team
        except ValueError:
            continue

    raise ValueError("Failed to create a valid team after many attempts.")

def create_valid_league(all_players, num_teams=5):
    max_attempts = 100
    for _ in range(max_attempts):
        random.shuffle(all_players)
        available_players = all_players.copy()
        used_names = set()
        teams = []

        try:
            for _ in range(num_teams):
                pool = [p for p in available_players if p.name not in used_names]
                team = create_valid_team_from_pool(pool)
                teams.append(team)
                used_names.update(p.name for p in team.players)
            return League(teams)
        except ValueError:
            continue

    raise ValueError("Failed to create a valid league after many attempts.")

def generate_population(players, num_leagues=5):
    population = []
    for _ in range(num_leagues):
        league = create_valid_league(players)
        population.append(league)
    return population


In [13]:
import numpy as np

In [16]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_standard_deviation_of_average_skills():.2f}")


--- League 1 ---
Alex Carter (GK) Skill: 85 Salary: €90.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Daniel Foster (DEF) Skill: 90 Salary: €110.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Spencer Ward (MID) Skill: 84 Salary: €85.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M
Tyler Jenkins (FWD) Skill: 80 Salary: €70.0M

Ryan Mitchell (GK) Skill: 83 Salary: €85.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Maxwell Flores (DEF) Skill: 81 Salary: €72.0M
Austin Torres (MID) Skill: 82 Salary: €80.0M
Nathan Wright (MID) Skill: 92 Salary: €120.0M
Landon Powell (FWD) Skill: 89 Salary: €110.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M

Blake Henderson (GK) Skill: 87 Salary: €95.0M
Lucas Bennett (DEF) Skill: 85 Salary: €90.0M
Mason Reed (DEF) Skill: 82 Salary: €75.0M
Gavin Richardson (MID) Skill: 87 Salary: €95.0M
Ashton Phillips (MID) Skill: 90 Salary: €110.0M
Zachary Nelson (FWD) Skill: 86 Salary: €92.0M
Elijah Sanders (FWD) Skill: 93 Salary: €140.0M

Jordan Smith (GK) Skill: 88 Sal

## Test Selection Functions

In [27]:
import random
from collections import Counter

def test_selection(population, selector_fn, selector_name, trials=100):
    picks = []
    for _ in range(trials):
        selected = selector_fn(population)
        # identify by fitness and index
        idx = population.index(selected)
        fitness = calculate_fitness(selected)
        picks.append((idx, round(fitness, 4)))
    counter = Counter(picks)

    print(f"\n{selector_name} results over {trials} trials:")
    for (idx, fit), count in counter.most_common():
        print(f"  League #{idx} (fitness={fit}): picked {count} times")


if __name__ == "__main__":
    # 1) Create a pool of, say, 50 players with random skills & costs
    positions = ["GK"]*10 + ["DEF"]*15 + ["MID"]*15 + ["FWD"]*10
    all_players = [
        Player(
            name=f"P{i}",
            position=random.choice(positions),
            skill=random.uniform(50, 100),
            cost=random.uniform(50, 200),
        )
        for i in range(1, 51)
    ]

### Roulette

In [33]:
test_selection(population, roulette_selection, "Roulette‐Wheel Selection")


Roulette‐Wheel Selection results over 100 trials:
  League #8 (fitness=0.5063): picked 19 times
  League #4 (fitness=0.6845): picked 18 times
  League #9 (fitness=1.1549): picked 10 times
  League #1 (fitness=0.5376): picked 9 times
  League #6 (fitness=1.6147): picked 9 times
  League #2 (fitness=1.0357): picked 9 times
  League #3 (fitness=0.8825): picked 8 times
  League #5 (fitness=0.7137): picked 8 times
  League #7 (fitness=1.3199): picked 7 times
  League #0 (fitness=1.0118): picked 3 times


### Tournament

In [ ]:
test_selection(
    population,
    lambda population: tournament_selection(population, tournament_size=3),
    "Tournament Selection (k=3)"
    )


Tournament Selection (k=3) results over 100 trials:
  League #1 (fitness=0.5376): picked 30 times
  League #8 (fitness=0.5063): picked 25 times
  League #5 (fitness=0.7137): picked 20 times
  League #4 (fitness=0.6845): picked 14 times
  League #3 (fitness=0.8825): picked 7 times
  League #2 (fitness=1.0357): picked 2 times
  League #0 (fitness=1.0118): picked 1 times
  League #9 (fitness=1.1549): picked 1 times


### Stochastic

In [56]:
from collections import Counter

def test_stochastic(population, selector_fn, selector_name, trials=100):
    counter = Counter()
    for _ in range(trials):
        selected_list = selector_fn(population)  # this is a list of num_parents Leagues
        for sel in selected_list:
            # find which original index this copy corresponds to
            idx = next(i for i, L in enumerate(population)
                       if calculate_fitness(L) == calculate_fitness(sel))
            counter[idx] += 1

    print(f"\n{selector_name} results over {trials} trials (total picks = {sum(counter.values())}):")
    for idx, count in counter.most_common():
        fit = round(calculate_fitness(population[idx]), 4)
        print(f"  League #{idx} (fitness={fit}): picked {count} times")


In [59]:
test_stochastic(
    population,
    lambda population: stochastic_selection(population, num_parents=5),
    "Stochastic Universal Sampling (N=5)",
    trials=100
)



Stochastic Universal Sampling (N=5) results over 100 trials (total picks = 500):
  League #8 (fitness=0.5063): picked 84 times
  League #1 (fitness=0.5376): picked 78 times
  League #4 (fitness=0.6845): picked 68 times
  League #2 (fitness=1.0357): picked 49 times
  League #5 (fitness=0.7137): picked 47 times
  League #0 (fitness=1.0118): picked 42 times
  League #3 (fitness=0.8825): picked 39 times
  League #6 (fitness=1.6147): picked 37 times
  League #7 (fitness=1.3199): picked 30 times
  League #9 (fitness=1.1549): picked 26 times
